### Importation de packages

In [ ]:
import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np
import pandas as pd
import os
import time

### Fonction pour télécharger les données mensuelles du vent

##### *Les données sont téléchargées depuis le site européen (https://cds.climate.copernicus.eu/). Cette plateforme met à disposition un package sur python pour faire des requêtes.*

In [ ]:
def telecharger_et_sauver_mensuel_era5(annee, mois, output_dir="./csv_vent_mensuel", pause_sec=60):

    """
    Cette fonction est responsable du téléchargement des données de vent à partir de l'API ERA5 et
    de leur sauvegarde dans un fichier CSV.

    paramètres :

    - annee et mois : période pour laquelle les données sont téléchargées.
    - output_dir : répertoire où le fichier CSV sera sauvegardé.
    - pause_sec : temps d'attente entre les requêtes pour éviter de surcharger l'API.

    """

    print(f"\n=== Traitement : {annee}-{mois:02d} ===")
    output_csv = os.path.join(output_dir, f"vent_{annee}_{mois:02d}.csv")

    # Sauter si déjà traité
    if os.path.exists(output_csv):
        print(f" Déjà existant, on saute : {output_csv}")
        return

    nc_path = f"vent_era5_{annee}_{mois:02d}.nc"
    client = cdsapi.Client()

    try:
        client.retrieve(
            'reanalysis-era5-single-levels',
            {
                "product_type": "reanalysis",
                "variable": [
                    "10m_u_component_of_wind",
                    "10m_v_component_of_wind",
                    "100m_u_component_of_wind",
                    "100m_v_component_of_wind",
                    "10m_u_component_of_neutral_wind",
                    "10m_v_component_of_neutral_wind",
                    "instantaneous_10m_wind_gust"
                ],
                "year": str(annee),
                "month": f"{mois:02d}",
                "day": [f"{d:02d}" for d in range(1, 32)],
                "time": [f"{h:02d}:00" for h in range(24)],
                "format": "netcdf",
                "area": [51.5, -5.5, 41.0, 9.5]  # France ( attention certaines zones hors de la France sont sélectionnées)
            },
            nc_path
        )
    except Exception as e:
        print(f" Erreur de téléchargement {annee}-{mois:02d} : {e}")
        return

    # Lecture NetCDF directement
    try:
        f = netCDF4.Dataset(nc_path)
    except Exception as e:
        print(f" Erreur d'ouverture du fichier .nc : {e}")
        return

    print(" Variables NetCDF :", list(f.variables.keys()))

    try:
        time_var_name = [k for k in f.variables if 'time' in k.lower()][0]
    except IndexError:
        raise KeyError(" Aucune variable temporelle détectée dans le NetCDF.")

    times = num2date(f.variables[time_var_name][:], f.variables[time_var_name].units)
    latitudes = f.variables['latitude'][:]
    longitudes = f.variables['longitude'][:]

    # Construction DataFrame de base
    times_grid, lat_grid, lon_grid = np.meshgrid(times, latitudes, longitudes, indexing='ij')
    df = pd.DataFrame({
        'time': [t.isoformat() for t in times_grid.flatten()],
        'latitude': lat_grid.flatten(),
        'longitude': lon_grid.flatten()
    })

    # Lecture des variables météo
    variables = {
        "u10": "u10", "v10": "v10",
        "u100": "u100", "v100": "v100",
        "u10n": "u10n", "v10n": "v10n",
        "gust": "i10fg"
    }

    data_arrays = {}
    for short, var in variables.items():
        if var in f.variables:
            print(f" Lecture : {var}")
            data = f.variables[var][:]
            df[short] = data.flatten()
            data_arrays[short] = data
        else:
            print(f" Variable absente : {var}")

    # Calcul des vitesses du vent
    def wind_speed(u, v):
        return np.sqrt(u**2 + v**2)

    if "u10" in data_arrays and "v10" in data_arrays:
        df["wind_speed_10m"] = wind_speed(data_arrays["u10"], data_arrays["v10"]).flatten()
    if "u100" in data_arrays and "v100" in data_arrays:
        df["wind_speed_100m"] = wind_speed(data_arrays["u100"], data_arrays["v100"]).flatten()
    if "u10n" in data_arrays and "v10n" in data_arrays:
        df["wind_speed_10m_neutral"] = wind_speed(data_arrays["u10n"], data_arrays["v10n"]).flatten()

    # Sauvegarde du fichier CSV
    os.makedirs(output_dir, exist_ok=True)
    df.to_csv(output_csv, index=False)
    print(f" Sauvegarde : {output_csv}")

    # Pause pour ne pas surcharger l'API
    time.sleep(pause_sec)


# Boucle 2012 à 2025 (on fait les boucles par morceaux de plage temporelle. 
# Par exemple on prend de 2012 à 2016 puis 2017 à 2020 et ainsi de suite)

for annee in range(2012, 2020):
    for mois in range(1, 13):
        try:
            telecharger_et_sauver_mensuel_era5(annee, mois)
        except Exception as e:
            print(f" Problème {annee}-{mois:02d} : {e}")


=== Traitement : 2012-01 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_01.csv

=== Traitement : 2012-02 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_02.csv

=== Traitement : 2012-03 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_03.csv

=== Traitement : 2012-04 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_04.csv

=== Traitement : 2012-05 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_05.csv

=== Traitement : 2012-06 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_06.csv

=== Traitement : 2012-07 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_07.csv

=== Traitement : 2012-08 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_08.csv

=== Traitement : 2012-09 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_09.csv

=== Traitement : 2012-10 ===
⏭ Déjà existant, on saute : ./csv_vent_mensuel\vent_2012_10.csv

=== Traitement : 2012-11 ===
⏭ Déjà existant, on saute : ./

2025-05-19 16:15:27,626 WARNING [2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
2025-05-19 16:15:27,626 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-19 16:15:27,626 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-19 16:15:28,147 INFO Request ID is 3a07a25b-f859-4ede-9725-771efbc6f856
2025-05-19 16:15:28,216 INFO status has been updated to accepted
2025-05-19 16:16:00,967 INFO status has been updated to running
2025-05-19 16:21:47,762 INFO st

 Variables NetCDF : ['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10', 'u100', 'v100', 'u10n', 'v10n', 'i10fg']
 Lecture : u10
 Lecture : v10
 Lecture : u100
 Lecture : v100
 Lecture : u10n
 Lecture : v10n
 Lecture : i10fg
 Sauvegarde : ./csv_vent_mensuel\vent_2019_10.csv

=== Traitement : 2019-11 ===


2025-05-19 16:24:15,669 WARNING [2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
2025-05-19 16:24:15,674 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-19 16:24:15,677 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-19 16:24:16,338 INFO Request ID is c1a5f9c2-2743-42c2-9329-21f18570b6e4
2025-05-19 16:24:16,418 INFO status has been updated to accepted
2025-05-19 16:26:10,715 INFO status has been updated to running
2025-05-19 16:30:35,833 INFO st

 Variables NetCDF : ['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10', 'u100', 'v100', 'u10n', 'v10n', 'i10fg']
 Lecture : u10
 Lecture : v10
 Lecture : u100
 Lecture : v100
 Lecture : u10n
 Lecture : v10n
 Lecture : i10fg
 Sauvegarde : ./csv_vent_mensuel\vent_2019_11.csv

=== Traitement : 2019-12 ===


2025-05-19 16:33:08,291 WARNING [2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
2025-05-19 16:33:08,291 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-19 16:33:08,306 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-19 16:33:09,358 INFO Request ID is 4af8e9ae-363f-44b0-aa35-7a2dafbac1c3
2025-05-19 16:33:09,421 INFO status has been updated to accepted
2025-05-19 16:33:18,554 INFO status has been updated to running
2025-05-19 16:37:29,125 INFO st

 Variables NetCDF : ['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10', 'u100', 'v100', 'u10n', 'v10n', 'i10fg']
 Lecture : u10
 Lecture : v10
 Lecture : u100
 Lecture : v100
 Lecture : u10n
 Lecture : v10n
 Lecture : i10fg
 Sauvegarde : ./csv_vent_mensuel\vent_2019_12.csv
